In [271]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Wgląd do danych

In [272]:
df = pd.read_csv('../original_data/14_WIELKOPOLSKIE.csv')
df

,Unnamed: 0,Dni od zakupu,Marka,Wiek kupującego,Płeć kupującego,Ocena
0,0,3,Electrolux,41.0,M,3.0
1,1,7,Electrolux,57.0,M,3.0
2,2,9,Electrolux,NaN,K,4.5
3,3,8,Beko,NaN,M,1.5
4,4,10,Beko,35.0,M,2.5
...,...,...,...,...,...,...
529,529,8,Samsung,39.0,M,1.5
530,530,9,Electrolux,26.0,K,1.5
531,531,6,Beko,43.0,M,2.5
532,532,5,Beko,NaN,M,2.5


In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       534 non-null    int64  
 1   Dni od zakupu    534 non-null    int64  
 2   Marka            534 non-null    object 
 3   Wiek kupującego  473 non-null    float64
 4   Płeć kupującego  534 non-null    object 
 5   Ocena            534 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 25.2+ KB


### Usunięcie niepotrzebnej kolumny z indeksami (pandas nadaje unikalne indeksy, więc mamy powtórzenie tych samych danych)

In [274]:
df.drop(columns='Unnamed: 0', inplace=True)
df

,Dni od zakupu,Marka,Wiek kupującego,Płeć kupującego,Ocena
0,3,Electrolux,41.0,M,3.0
1,7,Electrolux,57.0,M,3.0
2,9,Electrolux,NaN,K,4.5
3,8,Beko,NaN,M,1.5
4,10,Beko,35.0,M,2.5
...,...,...,...,...,...
529,8,Samsung,39.0,M,1.5
530,9,Electrolux,26.0,K,1.5
531,6,Beko,43.0,M,2.5
532,5,Beko,NaN,M,2.5


### Zmiana języka na angielski dla uniknięcia w danych występowania polskich znaków oraz spacji

In [275]:
column_names = {'Dni od zakupu': 'Days_From_Purchase',
                'Marka': 'Brand',
                'Wiek kupującego': 'Buyer_Age',
                'Płeć kupującego': 'Buyer_Sex',
                'Ocena': 'Rating'}

df.rename(columns=column_names, inplace=True)
df

,Days_From_Purchase,Brand,Buyer_Age,Buyer_Sex,Rating
0,3,Electrolux,41.0,M,3.0
1,7,Electrolux,57.0,M,3.0
2,9,Electrolux,NaN,K,4.5
3,8,Beko,NaN,M,1.5
4,10,Beko,35.0,M,2.5
...,...,...,...,...,...
529,8,Samsung,39.0,M,1.5
530,9,Electrolux,26.0,K,1.5
531,6,Beko,43.0,M,2.5
532,5,Beko,NaN,M,2.5


In [276]:
file_exists = os.path.exists('../analysis_data/Prepared_Data.csv')
if not file_exists:
    df.to_csv('../analysis_data/Prepared_Data.csv', index=False)

### Sprawdzenie ilości niepełnych wierszy w tabeli

In [277]:
df.isnull().sum()

Days_From_Purchase     0
Brand                  0
Buyer_Age             61
Buyer_Sex              0
Rating                 0
dtype: int64

In [278]:
percent_sex = df['Buyer_Sex'].value_counts()['bd.'] / (df.shape[0]) * 100
percent_age = df['Buyer_Age'].isna().sum() / (df.shape[0]) * 100

print(f'W kolumnie wieku kupującego jest {percent_age:.2f} % niepełnych wierszy')
print(f'W kolumnie płci kupującego jest {percent_sex:.2f} % niepełnych wierszy')

W kolumnie wieku kupującego jest 11.42 % niepełnych wierszy
W kolumnie płci kupującego jest 2.81 % niepełnych wierszy


### Do analizy niektórych zależności należy usunąć niepełne wiersze. Liczba niepełnych wierszy w płci kupującego jest tak mała, że nie wpływa na wyniki, natomiast w wieku kupującego brakuje ponad 10% danych, co jest już liczbą wpływającą na wynik

In [279]:
# czyszczenie braku danych w wieku kupującego
clean_df = df.dropna()

# czyszczenie braku danych w płci kupującego
i = clean_df[clean_df.Buyer_Sex == 'bd.'].index  # wyciągnięcie indeksów
clean_df = clean_df.drop(i)

clean_df

,Days_From_Purchase,Brand,Buyer_Age,Buyer_Sex,Rating
0,3,Electrolux,41.0,M,3.0
1,7,Electrolux,57.0,M,3.0
4,10,Beko,35.0,M,2.5
5,5,Beko,53.0,M,2.0
6,5,Electrolux,27.0,K,4.5
...,...,...,...,...,...
528,2,Dyson,39.0,M,3.0
529,8,Samsung,39.0,M,1.5
530,9,Electrolux,26.0,K,1.5
531,6,Beko,43.0,M,2.5


In [280]:
file_exists = os.path.exists('../analysis_data/Clean_Data.csv')
if not file_exists:
    clean_df.to_csv('../analysis_data/Clean_Data.csv', index=False)

### Zastąpienie wieku kupującego przedziałami wiekowymi

In [281]:
df_range_age = clean_df.copy()

bins= [18, 25, 35, 45, 55, 65, 110]
labels = ['18-24', '25-34','35-44','45-54','55-64','65+']
df_range_age['Buyer_Age_Group'] = pd.cut(df_range_age['Buyer_Age'], bins=bins, labels=labels, right=False)

# wyrzucenie wieku
df_range_age.drop(columns='Buyer_Age', inplace=True)

# zmiana kolejności kolumn
df_range_age = df_range_age[['Days_From_Purchase', 'Brand', 'Buyer_Age_Group', 'Buyer_Sex', 'Rating']]

df_range_age

,Days_From_Purchase,Brand,Buyer_Age_Group,Buyer_Sex,Rating
0,3,Electrolux,35-44,M,3.0
1,7,Electrolux,55-64,M,3.0
4,10,Beko,35-44,M,2.5
5,5,Beko,45-54,M,2.0
6,5,Electrolux,25-34,K,4.5
...,...,...,...,...,...
528,2,Dyson,35-44,M,3.0
529,8,Samsung,35-44,M,1.5
530,9,Electrolux,25-34,K,1.5
531,6,Beko,35-44,M,2.5


In [282]:
file_exists = os.path.exists('../analysis_data/Data_Group_Age.csv')
if not file_exists:
    df_range_age.to_csv('../analysis_data/Data_Group_Age.csv', index=False)

### Zastosowanie przedziałów do kolumny minionych dni od wystawionej opinii oraz opisu wystawionej oceny

In [283]:
df_range_days = df.copy()

# przedziały czasu od wystawionej opinii
bins = [0, 3, 5, 8, 15, 100]
labels = ['do 2 dni', 'do 4 dni', 'do tygodnia', 'do 2 tygodni', 'powyżej 2 tygodni']
df_range_days['Days_From_Purchase_Range'] = pd.cut(df_range_days['Days_From_Purchase'], bins=bins, labels=labels, right=False)

# przedziały czasu od wystawionej opinii
bins = [0, 2, 4, 6]
labels = ['Negatywna', 'Średnia', 'Pozytywna']
df_range_days['Rating_Group'] = pd.cut(df_range_days['Rating'], bins=bins, labels=labels, right=False)

# zmiana kolejności kolumn
df_range_days = df_range_days[['Days_From_Purchase', 'Days_From_Purchase_Range', 'Brand', 'Buyer_Age', 'Buyer_Sex', 'Rating', 'Rating_Group']]

# sortowanie danych
df_range_days = df_range_days.sort_values(by=['Days_From_Purchase', 'Rating'])

df_range_days

,Days_From_Purchase,Days_From_Purchase_Range,Brand,Buyer_Age,Buyer_Sex,Rating,Rating_Group
418,0,do 2 dni,Dyson,45.0,M,3.5,Średnia
239,1,do 2 dni,Dyson,43.0,M,1.0,Negatywna
59,2,do 2 dni,Samsung,33.0,M,1.5,Negatywna
88,2,do 2 dni,Beko,47.0,M,2.0,Średnia
116,2,do 2 dni,Dyson,53.0,M,2.0,Średnia
...,...,...,...,...,...,...,...
14,14,do 2 tygodni,Tefal,41.0,M,3.0,Średnia
185,14,do 2 tygodni,Beko,47.0,M,3.0,Średnia
199,15,powyżej 2 tygodni,Electrolux,44.0,K,1.5,Negatywna
8,15,powyżej 2 tygodni,Electrolux,50.0,M,3.0,Średnia


In [284]:
file_exists = os.path.exists('../analysis_data/Data_Group_Days_Delay_Rating.csv')
if not file_exists:
    df_range_days.to_csv('../analysis_data/Data_Group_Days_Delay_Rating.csv', index=False)